In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline

# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm       

from itertools import combinations 

In [ ]:
# setActions = ['a1']
# setResponse = [] #....
# setEffect = ['e1', 'e2'] #.... 

## This function reads the artificially genereted tables from the designed csv
def read_table(exp, row, freq):
    r1 = pd.Series(row)[3:7].tolist();
#     print(r1)
    r2 =pd.Series(row)[7:11].tolist();
#     print(r2)
    r3 = pd.Series(row)[11:15].tolist();
#     print(r3)
    frame = { 'r1': r1, 'r2': r2, 'r3':r3 } 
    tempTable = pd.DataFrame(frame).T
    # for each cells times a 'freq'
    tempTable = tempTable*freq
    return(tempTable)


## This function reads the artificially genereted tables from the designed csv
def create_one_response_table(response2effectTable, response):
    oneResponseTable = response2effectTable
    currentR = response

    oneResponseTable.loc['total'] = pd.Series(oneResponseTable.sum())
    oneResponseTable.loc['nonR'] = oneResponseTable.loc['total'] - oneResponseTable.loc[currentR]

    oneResponseTableWithTotal = oneResponseTable
    
    for other_response in myResponses:
        if other_response != currentR:
            oneResponseTable = oneResponseTable.drop(other_response)
    
    oneResponseTable = oneResponseTable.drop('total')
    return(oneResponseTable, oneResponseTableWithTotal)
#     print(oneResponseTable)

def checking_non_neg_assumptions(oneResponseTable, response2effectTableWithTotal):
    # Checking assumptions
    boolAssumptionTrue = True

    # Assumption 1: For each expected cell, the number > 0
    for aj in response2effectTableWithTotal.columns:
        if response2effectTableWithTotal.at['total', aj] == 0:
            boolAssumptionTrue = False


            
    
    # TODO 
#     if !boolAssumptionTrue:
#         print("NA")

    return(boolAssumptionTrue)


def checking_80perc_expected_assumptions(expected):
    # Assumption 2: At least 80% of expected cells >= 5 // test on the oneResponseTable
    # TO DISCUSS!!! VERY DIFFICULT, checking the full table or checking the one response table. 
    return((expected > 0).sum(1).sum() >= (0.8 * 8))

def chi2_cont_row(df):
    stat, p, dof, expected = chi2_contingency(df, correction=True)
#     print('dof=%d' % dof)
#     print(expected)

    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
#     print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

#     if abs(stat) >= critical:
#         print('Dependent (reject H0)')
#     else:
#         print('Independent (fail to reject H0)')


    # interpret p-value
    alpha = 1.0 - prob
#     print('significance=%.3f, p=%.3f' % (alpha, p))
#     if p <= alpha:
#         print('Dependent (reject H0)')
#     else:
#         print('Independent (fail to reject H0)')
    return stat, p, dof, expected 

def test_corrcoef():
    a = np.array([
        [1, 2, 3, 4],
        [1, 3, 1, 4],
        [8, 3, 8, 5],
        [2, 3, 2, 1]])

    r1, p1 = corrcoef(a)
    r2, p2 = corrcoef_loop(a)

    assert np.allclose(r1, r2)
    assert np.allclose(p1, p2)
    
def calculate_number_of_significant_edges(oneResponseTable, currentR):
    countEdgesDraw = 0
    countZeroEdgesDraw = 0
    countSigLessEdgesDraw = 0
    countSigMoreEdgesDraw = 0
    countdiffedges = 0
    sharededges = 0
    adjresidual = 2.57 
    dfgfilter = 0.8
    shortfull = table.iloc[0:3, 0:4]
    maxtable = max(shortfull.max())
    threshold = maxtable-(maxtable*dfgfilter)
    
    stat, p, dof, expected = chi2_cont_row(oneResponseTable)
    
    rtable = sm.stats.Table(oneResponseTable)     
#         print(rtable.resid_pearson)
#         print(rtable.standardized_resids)

    subtable = pd.DataFrame(rtable.standardized_resids)
#     print(subtable)
    
    expectedT = pd.DataFrame(expected, columns = oneResponseTable.columns, index = oneResponseTable.index)
#     print(newTable)
#     print(expectedT)

    for aj in oneResponseTable.columns:
        if (subtable.at[currentR,aj] < -adjresidual) or (subtable.at[currentR,aj] > adjresidual):
            countEdgesDraw +=1
            if(subtable.at[currentR,aj] < -adjresidual):
                countSigLessEdgesDraw += 1
            if (subtable.at[currentR,aj] > adjresidual) :
                countSigMoreEdgesDraw += 1
            if(oneResponseTable.at[currentR, aj] == 0):
                countZeroEdgesDraw +=1
#             if(oneResponseTable.at[currentR, aj] > threshold) and ((subtable.at[currentR,aj] <= adjresidual) or (subtable.at[currentR,aj] >= -adjresidual)):
#                 countdiffedges += 1
#             if(oneResponseTable.at[currentR, aj] <= threshold) and ((subtable.at[currentR,aj] > adjresidual) or (subtable.at[currentR,aj] < -adjresidual)):
#                 countdiffedges += 1
            if(oneResponseTable.at[currentR, aj] > threshold):
                sharededges += 1
    return(countEdgesDraw, countZeroEdgesDraw, countSigLessEdgesDraw, countSigMoreEdgesDraw, countdiffedges, sharededges)

In [ ]:
header_list = ["nEff", "nResp", "step", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12"]

filenumber = 1
n = 10
for filenumber in range(filenumber, n):
    file = 'Archive/pmf_tables_for_evaluation_'+str(filenumber)+'.csv'
    expFullTable = pd.read_csv(file, ',',  names=header_list, encoding='latin1')
    
    #print(expFullTable.head())

    columnRes = 'Responses'

    expFullTable.count()
    
    # create 'table' from CSV
    rowNumber = 1
    freq = 100
    test = read_table(expFullTable, expFullTable.iloc[rowNumber], freq)
    # print(test)
    
    myResponses = {"r1", "r2", "r3"}
    create_one_response_table(test, 'r1')
    
    #calculate_number_of_significant_edges(test, 'r1')
    countRow = 0 

    for i, row in expFullTable.iterrows():
        if(countRow % 1000 == 0):
            print(countRow)
            countRow+=1
    
        orgtable = read_table(expFullTable, row, freq)
        table = orgtable.copy()
    #     print("This is table")
    #     print(table)
    #     print("----")
    
        totNumEdgesOfOurs = 0
        totNumZeroEdgesOurs = 0
        totNumSigMoreEdgesOurs = 0
        totNumDiffEdges = 0
        totNumSharedEdges = 0
    
        bool_Assumption_all_responses = True;
    
        pvalues = list()
        numEdges = list()
        numZeroEdges = list()
        numSigMoreEdges = list()
        numDiffEdges = list()
        numSharedEdges = list()
    
        # For each response 
        for response in myResponses:
            oneResponseTable, oneResponseTableWithTotal = create_one_response_table(table, response)
    #         print("This is one response:")
    #         print(oneResponseTable)
        
        
            # Test assumptions
            bool_Assumption_Single_Response = checking_non_neg_assumptions(oneResponseTable, oneResponseTableWithTotal)
            if(bool_Assumption_Single_Response):
                stat, p, dof, expected = chi2_cont_row(oneResponseTable)
            
                if (checking_80perc_expected_assumptions(expected)):
                    pvalues.append(p)
                    num, numZero, numSigLess, numSigMore, numDiff, numShared = calculate_number_of_significant_edges(oneResponseTable, response)
                    numEdges.append(num)
                    numZeroEdges.append(numZero)
                    numSigMoreEdges.append(numSigMore)
                    numDiffEdges.append(numDiff)
                    numSharedEdges.append(numShared)
                
                else:
                    bool_Assumption_all_responses = False;
            
            else:
                ## Chi-test's assumptions do not hold. We do not compute anything
                bool_Assumption_all_responses = False;
            
      
        expFullTable.at[i, 'ourNumEdges'] = str('NA')
        expFullTable.at[i, 'ourNumZeroEdges'] = str('NA')
        expFullTable.at[i, 'ourNumPosEdges'] = str('NA')
        expFullTable.at[i, 'ourNumSigMoreEdges'] = str('NA')
        expFullTable.at[i, 'DiffEdges'] = str('NA')
        expFullTable.at[i, 'SharedEdges'] = str('NA')
        if(bool_Assumption_all_responses):
        
            p_adjusted_list = multipletests(pvalues, alpha=.05, method='bonferroni')
            for index, p_adjusted in enumerate(p_adjusted_list[0]):
    #             print(p_adjusted)
                if(p_adjusted):
                    totNumEdgesOfOurs += numEdges[index]
                    totNumZeroEdgesOurs += numZeroEdges[index]
                    totNumSigMoreEdgesOurs += numSigMoreEdges[index]
                    totNumDiffEdges += numDiffEdges[index]
                    totNumSharedEdges += numSharedEdges[index]
        
            expFullTable.at[i, 'ourNumEdges'] = str(totNumEdgesOfOurs)
            expFullTable.at[i, 'ourNumZeroEdges'] = str(totNumZeroEdgesOurs)
            expFullTable.at[i, 'ourNumPosEdges'] = str(totNumEdgesOfOurs - totNumZeroEdgesOurs)
            expFullTable.at[i, 'ourNumSigMoreEdges'] = str(totNumSigMoreEdgesOurs)
            expFullTable.at[i, 'DiffEdges'] = str(totNumDiffEdges)
            expFullTable.at[i, 'SharedEdges'] = str(totNumSharedEdges)
    #     else:
    #         expFullTable.at[i, 'ourNumEdges'] = str('NA')
    #         expFullTable.at[i, 'ourNumZeroEdges'] = str('NA')
    #         expFullTable.at[i, 'ourNumPosEdges'] = str('NA')
    #         expFullTable.at[i, 'ourNumSigMoreEdges'] = str('NA')
    
        dfgfilter = 0.8
        shortfull = table.iloc[0:3, 0:4]
        maxtable = max(shortfull.max())
        threshold = maxtable-(maxtable*dfgfilter)
        dfgNumEdges80 = shortfull[shortfull > threshold].count().sum();
        dfgNumEdges100 = (orgtable > 0).sum(1).sum();
        #     print(dfgNumEdges.tolist())
  
        expFullTable.at[i, 'dfgNumEdges80'] = dfgNumEdges80
        expFullTable.at[i, 'dfgNumEdges100'] = dfgNumEdges100
    
    # standard deviation of each column (effect) and the average for all four columns
    expFullTable['sdE1'] = expFullTable[['c1','c5','c9']].std(axis=1)
    expFullTable['sdE2'] = expFullTable[['c2','c6','c10']].std(axis=1)
    expFullTable['sdE3'] = expFullTable[['c3','c7','c11']].std(axis=1)
    expFullTable['sdE4'] = expFullTable[['c4','c8','c12']].std(axis=1)
    expFullTable['sda'] = expFullTable[['sdE1','sdE2','sdE3','sdE4']].mean(axis=1)
    # round sda to 4 digits
    expFullTable = expFullTable.round({'sda': 4})

    #print(expFullTable.loc[1])
    file2 = "Archive/pmf_tables_for_evaluation_res_80per_"+str(filenumber)+".csv"
    expFullTable.to_csv(file2, ",", encoding="latin1")